In [1]:
# PantryManagement_Program References (1) xlsx file with (3) separate worksheets to store data
## 1. Inventory
## 2. Recipes
## 3. Shopping History

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# creating function for #1 Option: "Check My Inventory"

def check_inventory():
    '''
    This function checks the current inventory of items.
    It reads the 'Inventory' sheet from a specified Excel file and prints the inventory.
    
    The function handles scenarios where the Excel file or the 'Inventory' sheet might be missing,
    displaying an appropriate error message in such cases.
    
    Returns:
        None
    '''
    try:
        inventory_df = pd.read_excel('PantryManager_Data.xlsx', sheet_name='Inventory')
        if inventory_df.empty:
            print('Inventory is empty.')
        else:
            print(inventory_df)
    except FileNotFoundError:
        print('The Excel file was not found. Please check the file path.')
    except pd.errors.XLRDError:
        print('The \'Inventory\' sheet is missing in the Excel file.')


In [4]:
# creating functions for #2 Option: "Update my inventory"
# Separate functions for adding or removing items from inventory

def update_inventory_for_meal(recipe_id, inventory_df, recipes_df):
    '''
    Updates the inventory based on a specified recipe. 
    It deducts the quantities of ingredients used in the recipe from the inventory.

    Parameters:
        recipe_id (str): The unique ID of the recipe.
        inventory_df (DataFrame): The DataFrame representing the current inventory.
        recipes_df (DataFrame): The DataFrame containing the recipes.

    Returns:
        DataFrame: The updated inventory DataFrame.
    '''
    try:
        recipe = recipes_df[recipes_df['Recipe ID'] == recipe_id]
        
        if recipe.empty:
            print(f"No recipe found with ID: {recipe_id}")
            return inventory_df

        for index, row in recipe.iterrows():
            ingredient = row['Ingredients']
            quantity_needed = row['Quantity per Ingredient']

            if ingredient in inventory_df['Item Name'].values:
                inventory_df.loc[inventory_df['Item Name'] == ingredient, 'Quantity'] -= quantity_needed
            else:
                print(f"Ingredient {ingredient} not found in inventory.")

    except KeyError as e:
        print(f"Key error: {e}. Please check the column names.")
    except Exception as e:
        print(f"An error occurred: {e}")

    return inventory_df

def update_inventory_for_groceries(purchased_items, inventory_df):
    '''
    Updates the inventory based on purchased grocery items.
    It adds the quantities of purchased items to the inventory, and also updates the price.

    Parameters:
        purchased_items (list of dicts): A list of dictionaries, each representing a purchased item.
                                         Each dictionary should have 'name', 'quantity', and 'price'.
        inventory_df (DataFrame): The DataFrame representing the current inventory.

    Returns:
        DataFrame: The updated inventory DataFrame.
    '''
    try:
        for item in purchased_items:
            if item['name'] in inventory_df['Item Name'].values:
                inventory_df.loc[inventory_df['Item Name'] == item['name'], 'Quantity'] += item['quantity']
                inventory_df.loc[inventory_df['Item Name'] == item['name'], 'Price'] += item['price']
            else:
                new_row = pd.DataFrame([item])
                inventory_df = pd.concat([inventory_df, new_row], ignore_index=True)

    except KeyError as e:
        print(f"Key error: {e}. Please check the keys in purchased_items and column names in inventory_df.")
    except Exception as e:
        print(f"An error occurred: {e}")

    return inventory_df


In [5]:
# creating functions for #3 Option: "Find a Meal" 
def find_meal(inventory_df, recipes_df):
    '''
    Suggests recipes based on the current inventory. 
    It checks each recipe in the recipes DataFrame against the inventory and suggests those that can be made with available ingredients.

    Parameters:
        inventory_df (DataFrame): The DataFrame representing the current inventory.
        recipes_df (DataFrame): The DataFrame containing the recipes.

    Returns:
        list: A list of suggested recipe names that can be made with the current inventory.
    '''
    try:
        inventory = {row['Item Name']: row['Quantity'] for index, row in inventory_df.iterrows()}

        suggested_recipes = []
        for recipe_id, recipe_group in recipes_df.groupby('Recipe ID'):
            can_make = True
            for index, row in recipe_group.iterrows():
                ingredient = row['Ingredients']
                quantity_needed = row['Quantity per Ingredient']
                if ingredient not in inventory or inventory[ingredient] < quantity_needed:
                    can_make = False
                    break
            if can_make:
                suggested_recipes.append(recipe_group['Recipe Name'].iloc[0])

        return suggested_recipes

    except KeyError as e:
        print(f"Key error: {e}. Please check the column names.")
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

In [6]:
# Creating functions for #4 Option: Get My Shopping List
def get_shopping_list():
    '''
    Generates and returns a shopping list based on the current inventory and par quantities.
    It checks each item in the inventory and if the current quantity is less than the par quantity,
    the item is added to the shopping list.

    The function handles scenarios where the Excel file or specific sheets might be missing,
    displaying an appropriate error message in such cases.

    Returns:
        list of tuples: A list where each tuple contains the item name and the quantity to purchase.
                        The quantity is the difference between the par quantity and the current quantity.
                        Returns an empty list if an error occurs.
    '''
    try:
        inventory_df = pd.read_excel('PantryManager_Data.xlsx', sheet_name='Inventory')
        
        shopping_list = []
        for index, row in inventory_df.iterrows():
            if row['Quantity'] < row['Par Quantity']:
                item_to_buy = (row['Item Name'], row['Par Quantity'] - row['Quantity'])
                shopping_list.append(item_to_buy)

        return shopping_list

    except FileNotFoundError:
        print('The Excel file was not found. Please check the file path.')
        return []
    except pd.errors.XLRDError:
        print('The \'Inventory\' sheet is missing in the Excel file.')
        return []
    except Exception as e:
        print(f'An unexpected error occurred: {e}')
        return []



In [7]:
# Creating functions for #5 Option: Analyze Pantry Data
def analyze_pantry_data():
    '''
    Analyzes pantry data by loading the inventory and providing insights such as total items,
    items below par quantity, and the most expensive item.

    This function handles scenarios where the Excel file or specific sheets might be missing,
    displaying appropriate error messages in such cases.

    Returns:
        dict: A dictionary containing various insights about the pantry data.
              Returns an empty dictionary if an error occurs.
    '''
    try:
        inventory_df = pd.read_excel('PantryManager_Data.xlsx', sheet_name='Inventory')

        # Total items in inventory
        total_items = len(inventory_df)

        # Items below par quantity
        below_par = inventory_df[inventory_df['Quantity'] < inventory_df['Par Quantity']]

        # Most expensive item
        most_expensive_item = inventory_df.loc[inventory_df['Price'].idxmax()]['Item Name']

        insights = {
            'Total Items': total_items,
            'Items Below Par Quantity': below_par[['Item Name', 'Quantity', 'Par Quantity']].to_dict('records'),
            'Most Expensive Item': most_expensive_item
        }

        return insights

    except FileNotFoundError:
        print('The Excel file was not found. Please check the file path.')
        return {}
    except pd.errors.XLRDError:
        print('The \'Inventory\' sheet is missing in the Excel file.')
        return {}
    except Exception as e:
        print(f'An unexpected error occurred: {e}')
        return {}



In [8]:
# Creating functions for #6. Update a Database
def update_database(inventory_df):
    '''
    Updates the Excel database with the current state of the inventory DataFrame.
    
    This function handles scenarios where the Excel file might not be found or writable,
    displaying appropriate error messages in such cases.

    Parameters:
        inventory_df (DataFrame): The DataFrame representing the current inventory.

    Returns:
        bool: True if the database was updated successfully, False otherwise.
    '''
    try:
        # Write the DataFrame to Excel
        inventory_df.to_excel('PantryManager_Data.xlsx', sheet_name='Inventory', index=False)
        print('Database updated successfully.')
        return True

    except FileNotFoundError:
        print('The Excel file was not found. Please check the file path.')
    except PermissionError:
        print('Permission denied: Unable to write to the Excel file. Please ensure it is not open in another program.')
    except Exception as e:
        print(f'An unexpected error occurred: {e}')

    return False


In [9]:
# Creating Basic User Interface to start and work backwards. Starting with the options available for user.
# Load Data
def load_data():
    try:
        inventory_df = pd.read_excel('PantryManager_Data.xlsx', sheet_name='Inventory')
        recipes_df = pd.read_excel('PantryManager_Data.xlsx', sheet_name='Recipes')  # Assuming there's a Recipes sheet
        # Load other sheets if necessary
        return inventory_df, recipes_df
    except Exception as e:
        print(f'Error loading data: {e}')
        return None, None  # Return None or empty DataFrame for each sheet

# Main Interface
def main():
    inventory_df, recipes_df = load_data()
    if inventory_df is None or recipes_df is None:
        print("Failed to load data. Exiting.")
        return

    while True:
        print("\n--- Pantry Manager ---")
        print("1. Check Inventory")
        print("2. Update Inventory")
        print("3. Analyze Pantry Data")
        print("4. Get Shopping List")
        print("5. Find a Meal")
        print("6. Save Changes to Database")
        print("7. Exit")
        choice = input("Enter your choice: ")

        if choice == '1':
            check_inventory(inventory_df)
        elif choice == '2':
            # Assuming there's a function to handle inventory updates
            # Example: update_inventory(inventory_df)
            pass
        elif choice == '3':
            insights = analyze_pantry_data(inventory_df)
            print(insights)
        elif choice == '4':
            shopping_list = get_shopping_list(inventory_df)
            print("Shopping List:", shopping_list)
        elif choice == '5':
            suggested_recipes = find_meal(inventory_df, recipes_df)
            print("Suggested Recipes:", suggested_recipes)
        elif choice == '6':
            if update_database(inventory_df):
                print("Changes saved successfully.")
            else:
                print("Failed to save changes.")
        elif choice == '7':
            break
        else:
            print("Invalid choice, please try again.")

if __name__ == '__main__':
    main()




--- Pantry Manager ---
1. Check Inventory
2. Update Inventory
3. Analyze Pantry Data
4. Get Shopping List
5. Find a Meal
6. Save Changes to Database
7. Exit
Suggested Recipes: ['Scrambled Eggs', 'Toast with Jam', 'Cheese and Crackers']

--- Pantry Manager ---
1. Check Inventory
2. Update Inventory
3. Analyze Pantry Data
4. Get Shopping List
5. Find a Meal
6. Save Changes to Database
7. Exit
Suggested Recipes: ['Scrambled Eggs', 'Toast with Jam', 'Cheese and Crackers']

--- Pantry Manager ---
1. Check Inventory
2. Update Inventory
3. Analyze Pantry Data
4. Get Shopping List
5. Find a Meal
6. Save Changes to Database
7. Exit


TypeError: get_shopping_list() takes 0 positional arguments but 1 was given

In [10]:
# to update git repository: got to terminal up top and click "new terminal"
# then enter -> git pull origin main (this updates the local folder first from anything new in github itself)
# then enter -> git commit -m 'Comment on changes made' 
# then enter -> git push